## test llama-index-retriever

In [1]:
from llama_index.graph_stores.neo4j import Neo4jPGStore

graph_store = Neo4jPGStore(
    username="neo4j",
    password="neo4j",
    url="bolt://localhost:7687",
)
vec_store = None

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The procedure has a deprecated field. ('config' used by 'apoc.meta.graphSample' is deprecated.)} {position: line: 1, column: 1, offset: 0} for query: "CALL apoc.meta.graphSample() YIELD nodes, relationships RETURN nodes, [rel in relationships | {name:apoc.any.property(rel, 'type'), count: apoc.any.property(rel, 'count')}] AS relationships"
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.AggregationSkippedNull} {category: UNRECOGNIZED} {title: The query contains an aggregation function that skips null values.} {description: null value eliminated in set function.} {position: None} for query: 'MATCH (n:`Chunk`)\nWITH collect(distinct substring(toString(n.`file_type`), 0, 50)) AS `file_type_values`,\n 

In [2]:
from llama_index.core.indices.property_graph import (
    ImplicitPathExtractor,
    SimpleLLMPathExtractor,
)
from llama_index.core import PropertyGraphIndex

In [3]:
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding

In [4]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
llm = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    model=os.getenv("AZURE_OPENAI_MODEL", "gpt-4-turbo"),
    deployment_name=os.getenv("AZURE_OPENAI_MODEL"),
    temperature=0.8,  # just for the pirate
)

In [6]:
embed_model = AzureOpenAIEmbedding(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    model=os.getenv("AZURE_OPENAI_EMBEDDER_MODEL", "text-embedding-ada-002"),
    deployment_name=os.getenv("AZURE_OPENAI_EMBEDDER_NAME"),
)

In [7]:
index = PropertyGraphIndex.from_existing(
    graph_store,
    embed_model=embed_model,
    kg_extractors=[
        ImplicitPathExtractor(),
        SimpleLLMPathExtractor(
            llm=llm,
            num_workers=4,
            max_paths_per_chunk=10,
        ),
    ],
    show_progress=True,
)

In [8]:
from llama_index.core.indices.property_graph import VectorContextRetriever

kg_retriever = VectorContextRetriever(
    index.property_graph_store,
    embed_model=embed_model,
    similarity_top_k=2,
    path_depth=1,
    # include_text=False,
    include_text=True,
)

In [9]:
nodes = kg_retriever.retrieve(
    "Give me all the policies that are related to the environment."
)
print(len(nodes))
for idx, node in enumerate(nodes):
    print(f">> IDX: {idx}, {node.get_content()}")

1
>> IDX: 0, Here are some facts extracted from the provided text:

Expanded coverage residential loan policy -> Provides coverage for -> Damage to improvements
Homeowners policy of title insurance -> Provides coverage for -> Damage to improvements

First American Title Insurance Company
Underwriting Bulletin 2010–19: Gas and Oil and Coal Exceptions in Pennsylvania
Page 2 of 2

The Coal Exception and Oil and Gas Exception Are Not To Be Removed -                                          Often we receive
requests to eliminate an exception for coal, oil, gas, or other mineral interest shown on a title
commitment solely because the instrument in question (typically an oil and gas lease) has been recorded
for many years and/or there are no visible signs of any activity on the subject premises. Neither of these
reasons is sufficient for removing an exception related to coal, oil, gas, or other mineral interest.
Nothing short of a recorded release of the instrument in question or a Court Orde